Importing libraries

In [177]:
import numpy as np
import pandas as pd

Pre-processing the data


In [178]:
df_amz = pd.read_csv('/content/amz_com-ecommerce_sample.csv',encoding="Latin-1")
df_flp = pd.read_csv('/content/flipkart_com-ecommerce_sample.csv')

In [179]:
df_amz.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [180]:
#Dropping unnecessary columns from the dataframes
df_flp = df_flp.drop(["uniq_id",	"crawl_timestamp",	"product_url", "product_category_tree",	"pid", "image",	"is_FK_Advantage_product",	"description",	"product_rating",	"overall_rating",	"product_specifications","brand"],axis=1)
df_amz = df_amz.drop(["uniq_id",	"crawl_timestamp",	"product_url", "product_category_tree",	"pid", "image",	"is_FK_Advantage_product",	"description",	"product_rating",	"overall_rating",	"product_specifications","brand"],axis=1)

In [181]:
df_flp.isna().sum().sum(), df_amz.isna().sum().sum()

(156, 0)

In [182]:
#Replacing NaN values in the retail_price column with mean of the column (We don't need to perform this in the amazon dataset as it does not have NaN values)
df_flp['retail_price'] = df_flp['retail_price'].fillna(df_flp['retail_price'].mean()) 

#Replacing NaN values in the discounted_price column with mean of the column
df_flp['discounted_price'] = df_flp['discounted_price'].fillna(df_flp['discounted_price'].mean())

#Replacing Negative discount prices with 0, and negative retail prices with their positive counterparts
for i in range(len(df_flp['discounted_price'])):
  if df_flp.loc[i,'discounted_price']<0:
    df_flp['discounted_price'][i] = 0
  if df_flp.loc[i,'retail_price']<0:
    df_flp['retail_price'][i] = abs(df_flp.loc[i,'retail_price'])
    
for i in range(len(df_amz['discounted_price'])):
  if df_amz.loc[i,'discounted_price']<0:
    df_amz['discounted_price'][i] = 0
  if df_amz.loc[i,'retail_price']<0:
    df_amz['retail_price'][i] = abs(df_flp.loc[i,'retail_price'])

<ipython-input-182-528bc3f7ace3>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_amz['retail_price'][i] = abs(df_flp.loc[i,'retail_price'])


In [188]:
df_flp.isna().sum().sum()

0

Merging the dataframes 

In [183]:
df_flp = df_flp.rename(columns={'product_name': 'product name in flipkart', 'retail_price': 'retail price in flipkart', 'discounted_price': 'discounted price in flipkart'})
df_amz = df_amz.rename(columns={'product_name': 'product name in amazon','retail_price': 'retail price in amazon', 'discounted_price': 'discounted price in amazon'})

In [184]:
print(len(df_flp))
len(df_amz)

20000


20000

In [185]:
#Maintaining a common column in both dataframes to merge them
df_flp['product_name'] = df_flp.loc[:, 'product name in flipkart'].str.lower()
df_amz['product_name'] = df_amz.loc[:, 'product name in amazon'].str.lower()

Final result in simple table format

In [186]:
merged_df = pd.merge(df_flp, df_amz, on='product_name')
merged_df = merged_df.drop(['product_name'], axis = 1)
merged_df

,product name in flipkart,retail price in flipkart,discounted price in flipkart,product name in amazon,retail price in amazon,discounted price in amazon
0,Alisha Solid Women's Cycling Shorts,999.0,379.0,Alisha Solid Women's Cycling Shorts,982,438
1,Alisha Solid Women's Cycling Shorts,999.0,379.0,Alisha Solid Women's Cycling Shorts,694,325
2,Alisha Solid Women's Cycling Shorts,999.0,379.0,Alisha Solid Women's Cycling Shorts,1198,602
3,Alisha Solid Women's Cycling Shorts,999.0,379.0,Alisha Solid Women's Cycling Shorts,1197,542
4,Alisha Solid Women's Cycling Shorts,999.0,379.0,Alisha Solid Women's Cycling Shorts,990,479
...,...,...,...,...,...,...
152783,Purple Women Heels,1099.0,899.0,PURPLE WOMEN HEELS,1084,1040
152784,Uberlyfe Large Vinyl Sticker,1190.0,595.0,UBERLYFE LARGE VINYL STICKER,1186,683
152785,We Witches Comfy Hues Women Wedges,899.0,899.0,WE WITCHES COMFY HUES WOMEN WEDGES,885,1059
152786,Stylistry Women Heels,1799.0,499.0,STYLISTRY WOMEN HEELS,1784,615


Final result in a simple form

In [187]:
inp = input("Enter product details: ").lower()
merged_df.iloc[merged_df[merged_df['product name in flipkart'].str.lower()==inp].index]

Enter product details: Uberlyfe Large Vinyl Sticker


,product name in flipkart,retail price in flipkart,discounted price in flipkart,product name in amazon,retail price in amazon,discounted price in amazon
152784,Uberlyfe Large Vinyl Sticker,1190.0,595.0,UBERLYFE LARGE VINYL STICKER,1186,683
